# <div style="text-align: center">18.335/6.337 Problem Set 1 - Floating Point and Norms</div>
## <div style="text-align: center">Due Fri, Feb/16 at 11:59 pm (office hours: Thu, Feb/15, 4–5 pm)</div>
### <div style="text-align: center">Created by Wonseok Shin</div>

### <div style="text-align: right">Solved by Renee Swischuk in collaboration with carmen ursachi and brad walcher</div>

$\newcommand{\Cmat}[2]{\mathbb{C}^{#1\times#2}}\newcommand{\Cvec}[1]{\mathbb{C}^{#1}}\newcommand{\Rmat}[2]{\mathbb{R}^{#1\times#2}}\newcommand{\Rvec}[1]{\mathbb{R}^{#1}}\newcommand{\null}{\mathrm{null}}\newcommand{\range}{\mathrm{range}}\newcommand{\rank}{\mathrm{rank}}\newcommand{\nullity}{\mathrm{nullity}}\newcommand{\sign}{\mathrm{sign}}\newcommand{\norm}[1]{\left\|#1\right\|}\newcommand{\abs}[1]{\left|#1\right|}\newcommand{\epsmach}{\epsilon_\mathrm{machine}}\newcommand{\log}{\mathrm{log}}\newcommand{\tanh}{\mathrm{tanh}}\newcommand{\l}{\lambda}\newcommand{\d}{\delta}$

## General instructions
- Please archive all files (this Jupyter notebook plus other files if any) into a single ZIP file and upload it to the course website, so that the grader can run the Jupyter notebook in the extracted folder without adding anything.  (This greatly simplifies the grader's job.)
- Some questions may require proofs or explanations, and there are a few options to submit them electronically.
    1. Ideally, write down your proofs or explanations in the corresponding sections of this Jupyter notebook.  You can use $\LaTeX$ commands for mathematical symbols, like $A = \hat{U} \Sigma V^*$, $\sum_{j = 1}^{r} \sigma_j u_j v_j^*$, and $\left[
\begin{array}{cc}
 1 & 2 \\
 3 & 4 \\
\end{array}
\right]$.
    2. If you are not familiar with $\LaTeX$ commands, you can use any word processors with equation editors.  If you choose to use this option, make sure to convert the word processor file into PDF before submission.
    3. If none of the above works for you, you can also submit a scan of your handwritten notes, but your handwriting must be clean and readable.
- The scores of bonus problems, if any, will be used to compensate the points deducted in regular problems at the end of the semester.  In other words, 
$$
\text{your total p-set score} = \min\left\{\sum_n\text{your $n$th p-set score $with$ bonus problems}, \sum_n\text{perfect $n$th p-set score $without$ bonus problems}\right\}.
$$

## Problem 1.  Floating-point peculiarity [4 pts]

In this problem, we will see some peculiar behaviors of floating-point numbers and try to understand them.

A few things to keep in mind:

- As discussed in class, the bit representation of IEEE double-precision floating point numbers is $S\,E_1 \cdots E_{11} M_1 \cdots M_{52}$, where
    - a single bit $S$ stores the sign,
    - 11 bits $E_1 \cdots E_{11}$ store the exponent information, and 
    - 52 bits $M_1 \cdots M_{52}$ store the mantissa (or significand) information.
- `eps()` in Julia returns the gap between 1 and the next larger number in the double-precision floating-point system.  
    - This is *not* the "machine epsilon" $\epsmach = \frac{1}{2}\beta^{-(t-1)}$ defined in Eq. (13.3) of T&B, but is twice as big, i.e., `eps()` $=2\epsmach$.
    - T&B's definition of machine epsilon is not universal: many other authors take the value of `eps()` itself (not a half) as machine epsilon, as shown [here](https://en.wikipedia.org/wiki/Machine_epsilon#Values_for_standard_hardware_floating_point_arithmetics).
- `bin()` in Julia is useful for obtaining the binary representation of integers.  For example, `bin(23)` returns `"10111"`.

(a) [0.5 pts] Find the bit representation of the value returned by `eps()`.  (Indicate the values of the sign bit $S$, exponent bits $E_1 \cdots E_{11}$, and the mantissa bits $M_1 \cdots M_{52}$.)

***Solution.***

bit representation of eps() - 0011110010110000000000000000000000000000000000000000000000000000

$S = 0$, $\qquad E_1,...,E_{11} = 01111001011$, $\qquad M_1,...,M_{52} = zeros(52)$

In [1]:
bits(eps())

"0011110010110000000000000000000000000000000000000000000000000000"

(b) [1 pt] Evaluate `1+eps()-1` and `2+eps()-2`.  Explain why you get different results.  (Hint: fundamental axiom of floating-point arithmetic.)

***Solution.***

The gap between numbers in the floating point system is relative to the size of the numbers. According to the book, the next consecutive number after 1 is $1+2^{-52} = 1+eps()$, so $1+eps()$ is able to be stored correctly in the system. Then when we subract 1 from it, we get the correct answer of $eps()$. On the other hand, the next consecutive number after 2 is twice the number following 1 and is equal to $2+2^{-52}*2 = 2 + 2^{-51}$. Since $eps() < 2^{-51}$, the system is incable of storing $2+eps()$ because in falls in between the gap of the two consecutive numbers : $2$ and $2 + 2^{-51}$. We learned in class that the system will always round down in ambiguous situations, thus, $2+eps()$ is rounded down to 2 and when we subtract 2 from it, we get 0, which is wrong. Another way to look at it is to use the fundamental axiom of floating point arithmetic which states "every operation of floating point arithmetic is exact up to a relative error of size at most $\epsilon_{machine}$". In other words, the error in floating point arithmetic scales with the size of the numbers. The statement $2+eps()-2$ has an error equal to the size of $\epsilon_{machine}$. The statement $1+eps()-1$ has an error that is half the size of this, i.e smaller than machine epsilon, which is negligible and thus produces the correct answer. 

In [25]:
a = 1+eps()-1
b = 2+eps()-2

println("1 + eps() - 1 = ", a, "\n", "2 + eps() - 2 = ", b)

println("eps() < 2^(-51): ", eps() < 2^(-51.))

1 + eps() - 1 = 2.220446049250313e-16
2 + eps() - 2 = 0.0
eps() < 2^(-51): true


(c) [0.5 pts] Evaluate `(2^-537.)^2` and `(2^-538.)^2`.  Explain why you get 0 from the latter.

***Solution.***

In class we determined that the range for denormalized floating point number numbers, $fl(x)$, was $$2^{-52}*2^{-1022} \leq |fl(x)| \leq (1-2^{-52})*2^{-1022}$$. 

The lower bound is $2^{-52}*2^{-1022} = 2^{-1074}$

$(2^{-538})^2 = 2^{-1076}$

$(2^{-537})^2 = 2^{-1074}$

$(2^{-538})^2 < 2^{-52}*2^{-1022}  = (2^{-537})^2$

$(2^{-538})^2$ is below the minimum value defined as the range for denormalized floating point numbers, thus, it cannot be represented in this system and is truncated to 0. On the other hand, $(2^{-537})^2$ is equal to this minimum value and is able to be stored accurately.  

In [26]:
println("(2^-537)^2 = ",(2^-537.)^2, "\n", "(2^-538)^2 = ",(2^-538.)^2)


(2^-537)^2 = 5.0e-324
(2^-538)^2 = 0.0


(d) [1 pt] (Taken from Exercise 13.2b of T&B)  The double-precision floating-point system includes many integers, but not all of them.  What is the smallest positive integer $n$ that does not belong to this system?  

***Solution.***

In the IEEE formula, double precision floating point numbers are represented as $$\frac{m}{\beta^t}\beta^e$$
where $t=53$, $\beta = 2$ and $1 \leq m \leq \beta^t$. The largest $m$ can be is $2^{53}$, so the smallest integer not belonging would be $n=\beta^t + 1 = 2^{53}+1$. There are 0's everywhere except the most and least significant bit, so that least significant bit will be truncated and the number will not be stored correctly. 

(e) [1 pt] (Taken from Exercise 13.2c of T&B)  For the answer $n$ of Part (d), detertmine whether or not each of $n+1$, $n+2$, $n+3$ belongs to the double-precision floating-point system.

***Solution***
$n+1 \in F$ since it is another power of 2 $\rightarrow n+1 = \beta^{53}+2 = \beta^{53} + \beta^1$. 

$n+2 \notin F$ since it involves another digit. In the same way as $n+1$, $n+3\in F$

Write Julia code that validates your analysis.  Write the code such that it also confirms $n$ indeed does not belong to the double-precision floating-point system, whereas $n-3$, $n-2$, $n-1$ do (as $n$ is the smallest that does not).  (Hint: `float(n)` produces the floating-point approximation of an integer `n`.)

In [30]:
println("Check if any of the following belong to the double precision floating point system\n")
println("n-3: ", float(2^(53)-2) == 2^(53)-2)
println("n-2: ", float(2^(53)-1) == 2^(53)-1)
println("n-1: ", float(2^(53)) == 2^(53))
println("n: ", float(2^(53)+1) == 2^(53)+1)
println("n+1: ", float(2^(53)+2) == 2^(53)+2)
println("n+2: ", float(2^(53)+3) == 2^(53)+3)
println("n+3: ", float(2^(53)+4) == 2^(53)+4)
println("n = ",bin(2^(53)+1))

Check if any of the following belong to the double precision floating point system

n-3: true
n-2: true
n-1: true
n: false
n+1: true
n+2: false
n+3: true
n = 100000000000000000000000000000000000000000000000000001


## Problem 2.  Accurate evaluation of mathematical functions [3 pts]

In this problem, we will devise ways to evaluate some mathematical functions on computers accurately.

(a) [1 pt] Evaluation of $f(x) = e^x - 1$ around $x = 0$.

- Explain the numerical problem you may experience during this evaluation.
- Devise a method to avoid this problem.  (Hint: $\tanh(x)$.)
- Implement your method in `f_new(x)` below.

***Solution.***

- Catastrophic cancellation states: "If $x \approx y$ and $|x − y| << |x| + |y|$, then the computed result $fl(x) − fl(y)$ is in general inaccurate"$^{[1]}$. The function $f(x)$ evaluates to 1 at $x = 0$ so when we try to evaluate this function around 0, we are subtracting two very close numbers which will likely be innacurate.

$[1]$ - http://www.ams.sunysb.edu/~jiao/teaching/ams526_fall12/lectures/lecture08.pdf

- The following shows the steps to compute a transformed version of the equation:
$$\begin{align}
e^x - 1 & = e^x * \frac{e^{-2x} -1}{e^{-2x} -1} - 1 *\frac{e^{-2x} -1}{e^{-2x} -1} \qquad \text{multiply by 1}\\
& = \frac{e^{-x} - e^{x}}{e^{-2x}-1}  - 1\\
& = \frac{(e^x+1)(e^{-x}-1)}{(e^{-x}+1)(e^{-x}-1)} - 1 \qquad \text{factor}\\
& = \frac{(e^x+1)}{(e^{-x}+1)} - 1\\
& = \frac{(e^x+1)}{(e^{-x}+1)} - \frac{(e^{-x}+1)}{(e^{-x}+1)} \qquad \text{common denominator}\\
& = \frac{e^x-e^{-x}}{e^{-x}+1} \\
& = \frac{2\sinh(x)}{\cosh(x)-\sinh(x)+1} \qquad \text{trig identity}\\
& = \frac{2}{-1+\coth{\frac{x}{2}}} \qquad \text{multiply by $\frac{\sinh}{\sinh}$ and simplify}\\
& = \frac{2}{-1+\frac{1}{\tanh{\frac{x}{2}}}}\\
& = \frac{2}{\frac{-\tanh{\frac{x}{2}}}{\tanh{\frac{x}{2}}}+\frac{1}{\tanh{\frac{x}{2}}}}\\
& = \frac{2 \tanh{\frac{x}{2}}}{-\tanh{\frac{x}{2} + 1}}  \qquad \text{to make a numerically safer function}
\end{align}$$

In [31]:
f_old(x) = exp(x) - 1
f_new(x) = 2*tanh(x/2)/(-tanh(x/2)+1)

f_new (generic function with 1 method)

### Validation of your implementation

Validate your implementation of `f_new(x)` by using the following code block that compares the numerical derivatives of `f_old(x)` and `f_new(x)` at `x = 0`.  Unlike `f_old(x)` that produces the completely wrong derivative 0.0, `f_new(x)` must produce a derivative close to the exact value 1.0.

In [32]:
x = 0.0
dx = eps() / 10
println("f_old'(0) =  $((f_old(x+dx) - f_old(x)) / dx)")
println("f_new'(0) =  $((f_new(x+dx) - f_new(x)) / dx)")

f_old'(0) =  0.0
f_new'(0) =  1.0


(b) [1 pt] Evaluation of $g(x) = \log(1+x)$ around $x = 0$.  ($\log$ is the natural logarithm.)

- Explain the numerical problem you may experience during this evaluation.  (Hint: Prob. 1(b).)
- Devise a method to avoid this problem.  (Hint: what is the relationship between $e^x-1$ and $\log(1+x)$?)
- Implement your method in `g_new(x)` below.

***Solution.***

- This function suffers from swamping$^{[1]}$. When $x$ is very close to 0, 1 is a much larger value relative to $x$. Therefore, when we add a very small $x$ to 1, it is truncated down to 1 and essentially is not added. What we need is a way to avoid adding a very small number to 1. To do this, we can see that this function is the inverse of the function from part a and can compute the transformation of this function as the inverse of our transformation from part a. 

$[1]$ - http://www.ams.sunysb.edu/~jiao/teaching/ams526_fall12/lectures/lecture08.pdf

- $log(1+x)$ is the inverse of $e^x - 1$. 
$$\begin{align}
y & = e^x - 1\\
y +1 & = e^x\\
log(y+1) & = x
\end{align}$$
Thus, we just need to take the inverse of our solution for part a
$$\begin{align*}
y & = \frac{2}{-1+\coth{\frac{x}{2}}}\\
-1+\coth{\frac{x}{2}} & = \frac{2}{y}\\
\coth{\frac{x}{2}} & = \frac{2}{y}+1\\
\frac{x}{2} & = acoth({\frac{2}{y}+1})\\
x & = 2 * acoth({\frac{2}{y}+1})\\
\end{align*}$$

In [7]:
g_old(x) = log(1+x)
g_new(x) = 2*acoth(2/x +1)

g_new (generic function with 1 method)

### Validation of your implementation

Validate your implementation of `g_new(x)` by using the following code block that compares the numerical derivatives of `g_old(x)` and `g_new(x)` at `x = 0`.  Unlike `g_old(x)` that produces the completely wrong derivative 0.0, `g_new(x)` must produce a derivative close to the exact value 1.0.

In [8]:
x = 0.0
dx = eps() / 10
println("g_old'(0) =  $((g_old(x+dx) - g_old(x)) / dx)")
println("g_new'(0) =  $((g_new(x+dx) - g_new(x)) / dx)")

g_old'(0) =  0.0
g_new'(0) =  1.0


(b) [1 pt] Evaluation of $h(x,y) = \sqrt{x^2+y^2}$.

- Explain at least one numerical problem that is different from those occurred in Parts (a) and (b) that you may experience during this evaluation.
- Devise a method to avoid this problem.
- Implement your method in `h_new(x,y)` below.

***Solution.***

- Catastrophic overflow states: "When multiplying two very large numbers $x$ and $y$, then $|xy|$ can cause unnecessary overflow"$^{[1]}$. In this problem, we have $x$ and $y$ being very large numbers. When we square them both, we get even larger numbers that cannot be represented accurately in the floating point system causing unnecessary overflow.

$[1]$ - http://www.ams.sunysb.edu/~jiao/teaching/ams526_fall12/lectures/lecture08.pdf
- The following shows how to transform the equation to avoid overflow
$$\begin{align*}
\sqrt{x^2+y^2} & = \sqrt{x^2(1+\frac{y^2}{x^2})}\\
& = x\sqrt{1+\left(\frac{y}{x}\right)^2}
\end{align*}$$

by dividing the two numbers before we square them, the size of the number is drastically reduced. 

### Validation of your implementation

Validate your implementation of `h_new(x,y)` by using the following code block that compares the values of `h_old(x,y)` and `h_new(x,y)`.  The value of `h_old(x,y)` does not make sense here, but the value of your `h_new(x,y)` must.

In [9]:
h_old(x,y) = sqrt(x^2+y^2)
h_new(x,y) = x*sqrt(1+(y/x)^2)

h_new (generic function with 1 method)

In [34]:
x = 1e200
y = 1e200
println("h_old(x,y) = $(h_old(x,y))")
println("h_new(x,y) = $(h_new(x,y))")

h_old(x,y) = Inf
h_new(x,y) = 1.414213562373095e200


## Problem 3.  Dual norm and $B$ such that $\left\| A B x\right\| =\left\| A\right\|  \left\| B\right\|  \left\| x\right\|$ [3 pts]

(Partly taken from Exercise 3.6 of Trefethen and Bau)

In this problem, we will prove step-by-step that for a given square matrix $A \in \mathbb{C}^{m\times m}$ and column vector $x \in \mathbb{C}^m$, there exists a square matrix $B$ such that $\left\| A B x\right\| =\left\| A\right\|  \left\| B\right\|  \left\| x\right\|$, where the matrix norm is induced by the vector norm.  (This fact will be used in proving the important Theorem 12.2 of Trefethen & Bau later in this course.)

The *dual norm* turns out to be a useful tool for the proof.  Let $\left\| \cdot \right\|$ be any vector norm on $\mathbb{C}^m$ with $m \ge 2$.  The dual norm $\left\| \cdot \right\|_*$ of the norm $\left\| \cdot \right\|$ is defined as

$$
\left\| x\right\|_* = \max_{v \neq 0} \frac{\left| x^* v\right|}{\left\| v \right\|} = \max_{\left\| v \right\|=1} \left| x^* v \right|.
$$

Note that the dual norm is defined in terms of $\left\| \cdot \right\|$.

Below, the proofs of some parts rely on the proofs of the previous parts.  If the previous parts are too difficult to prove, you are welcome to take those parts for granted without proof and use them in proving the next parts.

(a) [0.5 pts] Prove that $\left\| \cdot \right\|_*$ is indeed a norm.

***Proof.***
To prove $\left\| \cdot \right\|_*$ is a norm, we need to prove 3 things:

1) $\left\| x \right\|_* \geq 0$ and $\left\| x \right\|_* = 0 \iff x = 0$

$\left\| x \right\|_*  = \displaystyle\max_{\left\| v \right\|=1} \left| x^* v \right|$, by definition, the absolute value is non negative thus, the result of this maximum must also be nonnegative. We have restricted $v$ to be nonzero, so the only way this can evaluate to 0 is exactly when $x = \vec{0}$. Therefore, 1) is true.

2) $\left\| x + y \right\|_* \leq \left\| x \right\|_* + \left\| y \right\|_*$

$
\begin{align*}
\left\| x + y \right\|_* &= \displaystyle\max_{\left\| v \right\|=1} \left| (x+y)^* v \right| = \displaystyle\max_{\left\| v \right\|=1} \left| (x^*+y^*) v \right|\\
& = \displaystyle\max_{\left\| v \right\|=1} \left| x^*v+y^*v \right|\\
& \leq \displaystyle\max_{\left\| v \right\|=1} \left| x^*v \right| + \left| y^*v \right| \qquad \rightarrow \text{since absolute value is a norm: }\left| a+b \right| \leq \left| a \right| + \left| b \right|\\
& = \displaystyle\max_{\left\| v \right\|=1} \left| x^*v\right| + \displaystyle\max_{\left\| v \right\|=1} \left| y^*v\right|\\
& = \left\| x \right\|_* + \left\| y \right\|_* 
\end{align*}
$

So 2) is true


3) $\left\| ax \right\|_* = |a| \left\| x \right\|_*$

$\begin{align*}
\left\| ax \right\|_* &= \displaystyle\max_{\left\| v \right\|=1} \left| (ax)^*v\right|\\
& = \displaystyle\max_{\left\| v \right\|=1} \left| ax^*v\right|\\
& = |a| \displaystyle\max_{\left\| v \right\|=1} \left| x^*v\right| \qquad \text{max does not depend on a}\\
& = |a| \left\| x \right\|_*
\end{align*}$


(b) [0.5 pts] Prove that $\left\| x\right\|_{**} \leq \left\| x\right\|$ for all $x\in \mathbb{C}^m$, i.e., the dual norm of the dual norm is less than or equal to the original norm.

***Proof.***

$\begin{align*}
\left\| x\right\|_{**} = \displaystyle\max_{ y \neq 0} \frac{\left| x^*y\right|}{\left\| y\right\|_{*}}\\
\end{align*}$

Then we can define:
$\begin{align*}
\left\| y\right\|_{*} & = \displaystyle\max_{ u \neq 0} \frac{\left| y^*u\right|}{\left\| u\right\|} \geq \frac{|y^*x|}{\left\| x\right\|} \qquad \text{since $u$ is the vector that maximizes the equation, using the vector $x$ must be less than or equal to the max value.}\\
\left\| x\right\| &\geq \frac{\left| y^*x\right|}{\left\| y\right\|_{*}}\\
\displaystyle\max_{ y \neq 0}\left\| x\right\| & \geq \displaystyle\max_{ y \neq 0} \frac{\left| y^*x\right|}{\left\| y\right\|_{*}} \qquad \text{taking the max of both sides wrt y}\\
\left\| x\right\| &\geq \left\| x\right\|_{**} \qquad \text{by definition}
\end{align*}$

(c) [0.5 pts] In fact, we have not only $\left\| x\right\|_{**} \leq \left\| x\right\|$ but also $\left\| x\right\|_{**} =\left\| x\right\|$ for all $x\in \mathbb{C}^m$, i.e., the dual norm of the dual norm *is* the original norm.  Using this fact without proof, show that for a given $x\in \mathbb{C}^m$, there exists a nonzero $w\in \mathbb{C}^m$ such that $w^* x = \left\| w\right\|_* \left\| x\right\|$.

***Proof.***

$
\begin{align*}
\left\| x\right\| & = \left\| x\right\|_{**} = \displaystyle\max_{ v \neq 0} \frac{\left| x^*v\right|}{\left\| v\right\|_{*}} = \frac{\left| x^*u\right|}{\left\| u\right\|_{*}} \qquad \text{assuming u maximizes}\\
\left\| x\right\| & = \frac{\left| x^*u\right|}{\left\| u\right\|_{*}}\\
\left\| x\right\|\left\| u\right\|_{*} & = \left| x^*u\right| = \left| u^*x\right|\\
\left\| x\right\|\left\| e^{i\theta}w\right\|_{*} & = \left| (e^{i\theta}w)^*x\right| \qquad \text{replace with equivalent complex form since $u \in\mathbb{C}^m$} \\
e^{i\theta}\left\| x\right\|\left\| w\right\|_{*} & = \left| e^{i\theta}w^*x\right| \\
e^{i\theta}\left\| x\right\|\left\| w\right\|_{*} & = e^{i\theta} \left| w^*x\right|\qquad \text{cancel imaginaries}\\
\left\| x\right\|\left\| w\right\|_{*} & = \left| w^*x\right| = w^*x \qquad \text{since w is real}
\end{align*}$

(d) [1 pt] Let nonzero $x,y\in \mathbb{C}^m$ be given.  Show that there exists a rank-1 matrix $B = y z^*$ such that $B x = y$ and $\left\| B\right\|  \left\| x\right\| =\left\| y\right\|$, where $\left\| B\right\|$ is the matrix norm of $B$ induced by the vector norm $\left\| \cdot \right\|$.  (Hint: use Part (c).)

***Proof.***

We need to prove that there exists a $z$ where (1)$B = yz^*$ such that (2)$Bx = y$ and (3)$\left\| B\right\|  \left\| x\right\| =\left\| y\right\|$

$\begin{align*}
Bx &= y \\ (yz^*)x &= y \qquad  \text{subbing (1) into (2)}\\
z^*x &= 1
\end{align*}$

Looking at (3), we can sub in (1) to obtain
$\left\| B\right\|  \left\| x\right\| =\left\| y\right\| \Rightarrow \left\| yz^*\right\|  \left\| x\right\| =\left\| y\right\|$ but we cannot split the norm of $yz^*$, so this is a dead end for us. Instead, consider the matrix norm of B.

$\begin{align*}
\left\| B\right\| &= \displaystyle \max_{v\neq 0} \frac{\left\| Bv\right\|}{\left\| v\right\|}\\
& =  \displaystyle \max_{v\neq 0} \frac{\left\| (yz^*)v\right\|}{\left\| v\right\|} = \displaystyle \max_{v\neq 0} \frac{\left\| y(z^*v)\right\|}{\left\| v\right\|} \qquad z^*v \text{ is a scalar so we can remove it from the norm by definition}\\
& = \displaystyle \max_{v\neq 0}|z^*v| \frac{\left\| y\right\|}{\left\| v\right\|}\\
& = \left\| y\right\|\displaystyle \max_{v\neq 0} \frac{|z^*v|}{\left\| v\right\|} \qquad \text{remove $y$ since the max does not depend on $y$}\\
& = \left\| y\right\|\left\| z\right\|_*
\end{align*}$

So now;

$\begin{align*}
\left\| B \right\|\left\| x\right\|  &= \left\| y\right\|\left\| z\right\|_*\left\| x\right\|\\
\end{align*}$

From part c we know that there exists a $z$ such that $z^*x = \left\| z\right\|_* \left\| x\right\|$. So we can plug in the result from above $\left\| z\right\|_*\left\| x\right\| = z^*x = 1 $ leading to :

$\begin{align*}
\left\| B \right\|\left\| x\right\|  &= \left\| y\right\|\left\| z\right\|_*\left\| x\right\| =  \left\| y\right\|z^*x = \left\| y\right\| \\
\left\| B \right\|\left\| x\right\|  &= \left\| y\right\|
\end{align*}$

Thus the conditions hold.
 



(e) [0.5 pts] For a given square matrix $A\in \mathbb{C}^{m\times m}$ and column vector $x\in \mathbb{C}^m$, show that there exists $B\in \mathbb{C}^{m\times m}$ such that $\left\| A B x \right\| =\left\| A \right\| \left\| B\right\| \left\| x\right\|$, where the matrix norm is induced by the vector norm.  (Hint: use Part (d).)  

***Proof.***

From part d we know that given $x$ and $y$, there exists a $B$ such that $Bx = y$.

The matrix norm of A:

$\begin{align*}
\left\| A\right\| &= \displaystyle \max_{v \neq 0} \frac{\left\| Av\right\|}{\left\| v\right\|} = \frac{\left\| Ay\right\|}{\left\| y \right\|} \qquad \text{assuming y is the value that maximizes}\\
\left\| A\right\|\left\| y \right\| &= \left\| Ay\right\|\\
\left\| Ay\right\| &= \left\| A\right\|\left\| y \right\| \qquad \text{flip the equation}\\
\left\| A(Bx)\right\| &= \left\| A\right\|\left\| B \right\|\left\| x \right\| \qquad \text{using part d : $\left\| B\right\|  \left\| x\right\| =\left\| y\right\|$ on the rhs and plugging in $Bx = y$ on the lhs}\\
\left\| ABx\right\| &= \left\| A\right\|\left\| B \right\|\left\| x \right\|
\end{align*}$

## Bonus Problem.  Accurate evaluation of an infinite series [2 pts]

Consider an infinite series $s = \sum_{k=1}^{\infty} \frac{1}{k^4}$.  The goal of this problem is to find the approximate value $\hat{s}$ of $s$ with double-precision accuracy (i.e., $\hat{s} = \mathrm{fl}(s)$).

The terms in the series decrease quickly to 0.  Therefore, the truncated series $s_n = \sum_{k=1}^{n} \frac{1}{k^4}$ for a finite $n$ would be a good approximation of the infinite series $s$ for sufficiently large $n$.

(a) [1 pt] Find $n$ for which $s_n$ is a double-precision-or-more accurate approximation of $s$.  Use the following hints:
- The error is $s - s_n = \sum_{k=n+1}^{\infty} \frac{1}{k^4}$.  How small must this error be to achieve the desired accuracy in $s_n$?
- What is a close upper bound of the error $s - s_n$?  Can you find one as the area under some curve, which can be calculated by a definite integral?

***Solution.***

We want $s - s_n = \sum_{k=n+1}^{\infty} \frac{1}{k^4}$ to be double precision accurate, which means we need the error to be less than floating point precision accuracy (i.e less than $2^{-53}$). In other words,

$s - s_n = \sum_{k=n+1}^{\infty} \frac{1}{k^4} \leq  2^{-53}$

We can turn this into an integral to solve for $n$

$
\begin{align*}
\displaystyle\int_{n+1}^{\infty} \frac{1}{k^4} &\leq 2^{-53}\\
\\
\displaystyle\int_{n+1}^{\infty} \frac{1}{k^4} & = -\frac{1}{3k^3}\biggr\rvert_{n+1}^{\infty}\\
& = \left(\frac{1}{3(n+1)^3}\right) \leq 2^{-53}\\
& = (n+1)^{-3} \leq 3 \cdot 2^{-53}\\
& = n+1 \leq (3\cdot 2^{-53})^{-\frac{1}{3}}\\
& = n\leq 144263.40278903875
\end{align*}$

$n = 144264$ will provide double precision accuracy. 

In [38]:
(3*2^(-53.))^(-1/3.)

144263.40278903875

(b) [1 pt] In fact, $s = \frac{\pi^4}{90}$ exactly (see [here](https://en.wikipedia.org/wiki/Riemann_zeta_function#Specific_values)), so the desired $\hat{s}$ is nothing but the floating-point representation of $\frac{\pi^4}{90}$. 

Now, the function `s_truncated` below takes `n` as an argument and evaluates the truncated series $s_n$.  You may expect that `s_truncated` would return $\hat{s}$ if $n$ obtained in Part (a) is used as the argument, but you can verify that is not the case using the subsequent code block.

Change one line of `s_truncated` to make it evaluate $s_n$ more accurately.  Briefly explain in words why your change makes `s_truncated` more accurate.

After this change, `s_truncated` for $n$ obtained in Part (a) must return $\hat{s}$.

***Solution.***

When we sum from 1 to n we add numbers in the following order:

$1 + 2^{-4} + 3^{-4}+...+ n^{-4}$

These numbers are getting smaller and smaller. For example $n^{-4}$ is 5 orders of magnitude smaller than $eps()$. As an example below, when we add $n^{-4} + eps()$, the result is $eps()$. In other words, $n^{-4}$ isnt even getting added to $eps()$ because it is so small relative to $eps()$. When we start our running total at 1, we start with a number that is huge relative to the numbers we will be adding to it later in the loop. Eventually these numbers will become so small that they will no longer be added to the running total and the precision will be lost. To fix this, we have reversed the indexing of the loop so that our total begins with the highest precision numbers and the total grows with the size of the numbers. This will allow all values to be added to the sum and produces an appropriate error value. 

In [36]:
function s_truncated(n)
    s = 0.0
    for k = n:-1:1
        s += 1.0/float(k)^4
    end
    
    return s    
end

s_truncated (generic function with 1 method)

In [44]:
n = 144264  # use your n obtained in Part (a)
s_approx = s_truncated(n)
s = π^4 / 90
println("error:",s - s_approx)

println(n^(-4.)+eps())
println(n^(-4.))

error:0.0
2.2204691362831835e-16
2.308703287061033e-21
